## 6.3. Dog-Cat classification task and Transfer Learning(来自 CNN-Tutorial)   

+ 使用ImageFolder加载数据   
+ 使用ResNet做迁移学习

如我们从讲座中学到的那样，在许多场景中我们希望重用一个已经训练过的模型，用于一个类似的任务(称之为target task)，因为   
+ 原始模型使用大量数据(well-resource)进行训练，但在target task我们不具备这种条件。也许我们只有很少的(好的)数据用于目标任务(low-resource)，或者用于target task的数据位于原始任务的其他领域，或者两者都有……
+ 也许原来的模型已经很好了，我们不想使用相同的架构，并且从零开始为target task训练它，可能从原始模型中提取的特征(features extracted from)对新target task有很好的效果。   

在新的类似任务上使用已经训练过的模型进行任务（宽松地）称为**“迁移学习”**。 迁移学习是深度学习和机器学习中的重要概念。 

### 6.3.1. Dog-Cat classification task
首先，我们应该查看与CIFAR10类似的任务：Dog-Cat分类任务。 回顾CIFAR10任务：我们需要将32×32的彩色图像分类为10个不同的类别：飞机，汽车，鸟类，猫，鹿，狗，青蛙，马，船和卡车。 我们的Dog-Cat分类任务相似，但更为简单。它将在单独的数据集DogsvsCats上进行训练，该数据集是我从受欢迎的Kaggle的[Kaggle's DogsvsCat challenge](https://www.kaggle.com/c/dogs-vs-cats)中获取的(由于原始数据集用作这个challenge的数据，因此我没有test set的真实标签，所以下面我们将一些训练集分为了验证集和测试集-仅用于演示目的)。   
与CIFAR10任务相比，此处的主要区别在于图像的分辨率不同：大多数图像不是正方形图像，并且大多数图像的分辨率比32×32大得多。 在馈入我们的网络之前，**我们需要将它们scale为32×32**。

首先，您可以在此处下载zip文件：http://i13pc106.anthropomatik.kit.edu/~tha/teaching/dogs-vs-cats.zip。 然后将其上传到数据文件夹。 然后取消注释并运行以下命令以将其解压缩：
+ 下载不了，我单独在kaggle下载了train的zip数据放到了目录下解压缩
+ 训练之后(完成这个练习)将会把这个数据移动到数据仓库防止占用同步空间

In [1]:
import zipfile
with zipfile.ZipFile("Data/dogs-vs-cats.zip","r") as zip_ref:
    # Extract to storage/ of Gradient (free persistent storage)
    zip_ref.extractall("storage/Data")

1. 构建自定义数据集custom dataset:
  + 查看文件目录知道图片是以这种名称保存的`cat.6089.jpg`
  + 我们现在要提取出label  
  
  
`label = int(path.split('.')[0] == 'dog')` 是指在`train=True`的情况下如果图片的名称是'dog'标签是1，如果不是(图像是cat)标签是0；否则所有图片的标签都是-1

In [1]:
import numpy as np
import os
from PIL import Image

# One-hot encoding
def one_hot_encoding(idx):
    one_hot_array = np.zeros(10)
    one_hot_array[idx] = 1
    return one_hot_array

# 解析raw数据函数-调用这个函数将返回image(PIL格式)和image的label(int，或者np.array)
def parse_data(data_dir, path, train=True, one_hot=False):
    # path 是指对任意一个path的图片
    label = int(path.split('.')[0] == 'dog') if train else -1
    # 把label独热编码
    if one_hot:
        label = np.array(map(one_hot_encoding, label))
        
    # 再返回PILimage 典型路径为 storage/Data/cat.6089.jpg
    image = Image.open(os.path.join(data_dir, path))
    return image, label 

+ 先看使用pytorch自带的数据集的加载方法   
```python
train_dataset = torchvision.datasets.CIFAR10(root='Data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)
```
> 实现：我们可以自己去到cifar10的官网上把CIFAR-10 python version(http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz) 下载下来，然后解压为cifar-10-batches-py文件夹，并复制到相对目录`root=`下。(若设置download=True，则程序会自动从网上下载cifar10数据到相对目录`root=`下，但这样小伙伴们可能要等一个世纪了)，并对训练集进行加载(train=True)。

下面简单讲解`root`、`train`、`download`、`transform`这四个参数

1.`root`，表示cifar10数据的加载的相对目录   
2.`train`，表示是否加载数据库的**训练集**，false的时候加载测试集   
3.`download`，表示是否自动下载cifar数据集   
4.`transform`，表示是否需要对数据进行预处理，None为不进行预处理

+ **自己编加载数据集的函数，加载数据的用法**   
```python
train_dataset = MyDogCatData(main_dir,
                        data_type="train",
                        transform=transform)
```   


+ 另外使用通用的数据加载器 ImageFolder   
> 
```python
data_dir = r'D:\Jupyterlab_data\Cat_Dog_data\train' 
# TODO: create the ImageFolder
dataset = datasets.ImageFolder(data_dir, transform=transform)
```

加载自定义数据集：当我们需要加载自己的数据集的时候也可以借鉴这种方法，只需要继承`torch.utils.data.Dataset`类并重写`__init__`,`__len__`,以及`__getitem__`这三个方法即可。这样组着的类可以直接作为参数传入到`torch.util.data.DataLoader`中去。   

例如CIFAR包含的构造函数的参数是：
> https://pytorch.org/docs/stable/_modules/torchvision/datasets/cifar.html#CIFAR10   
```python
 """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.

Args:
root (string): Root directory of dataset where directory ``cifar-10-batches-py`` exists or will be saved to if download is set to True.

train (bool, optional): If True, creates dataset from training set, otherwise creates from test set.
        
transform (callable, optional): A function/transform that takes in an PIL image and returns a transformed version. E.g, ``transforms.RandomCrop``
        
target_transform (callable, optional): A function/transform that takes in the target and transforms it.
        
download (bool, optional): If true, downloads the dataset from the internet and puts it in root directory. If dataset is already downloaded, it is not downloaded again.
    """
```

In [3]:
import torch 
from torch.utils.data import Dataset

class MyDogCatData(Dataset): ## 仿照class DatasetFolder(data.Dataset):
    """
    写文本说明
    Args:
        main_dir (string): Directory with all the images.
        data_type ("train|valid|test"): is it training/validation/testing data
        transform (callable, optional): Optional transform to be applied on a sample.
    """
    
    def __init__(self, main_dir, data_type='train', transform=None, one_hot=False):
        # 必须指定transform
        self.transform = transform
        
        # self.train 是true还是false >>> # training set or test set
        # >> 没太理解有啥用 见下说明
        self.data_type = data_type
        if self.data_type == 'train' or self.data_type == 'valid':
            self._train = True
        elif self.data_type == 'test':
            self._train = False
        else:
            self._train = None
            raise NameError('DataType (train|valid|test) provided incorrect. No data loaded!')
            
        # 读取数据 data_dir 实现读取这个路径 字符串'\storage\Data\train'
        # 返回[(image1, label1),(...)]
        self.data = []
        data_dir = os.path.join(main_dir, self.data_type)
        
        #self.data = [parse_data(data_dir, path, train=self._train) for path in os.listdir(data_dir)]
        for path in os.listdir(data_dir):
            image, label = parse_data(data_dir, path, train=self._train)
            image = self.transform(image)
            self.data.append((image, label))
        
        # 对数据执行transform/见下说明
        # >>> 修正在for循环读取image时同步transform
        # 举个transform的例子 self.transform=transforms.Reseze((32,32))
        # 开始写错为 [self.transform(image, label)
        
        # self.data = [(self.transform(image), label) for (image, label) in self.data]
        
    def __len__(self):
        return len(self.data)
    
    # 如果在类中定义了__getitem__()方法，那么其实例对象dataset的（假设为P）
    # 就可以这样P[key]取值。当实例对象做P[key]运算时，就会调用类中的__getitem__(key)方法
    # 用来根据index查看数据可，需要注意的是 数据在生成实例的时候一般会指定transform
    # 注意辨析
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, label) where image is class_index of the label class.
        """
        # 得到image和其所属的label
        # self.data 是一个元组列表
        image, label = self.data[index]
        return image, label
    
    

说明：

```python
self.data = []
        data_dir = os.path.join(main_dir, self.data_type)
        
        #self.data = [parse_data(data_dir, path, train=self._train) for path in os.listdir(data_dir)]
        for path in os.listdir(data_dir):
            image, label = parse_data(data_dir, path)
            self.data.append((image, label))
        self.data = [(self.transform(image), label) for (image, label) in self.data]
```
报错：OSError: [Errno 24] Too many open files:
> ---> 34             image, label = parse_data(data_dir, path)
> 以restart 确实存在

解决方法：
```python
#self.data = [parse_data(data_dir, path, train=self._train) for path in os.listdir(data_dir)]
        for path in os.listdir(data_dir):
            image, label = parse_data(data_dir, path, train=self._train)
            image = self.transform(image)
            self.data.append((image, label))
```

+ 将transform写在 for循环中，totensor起作用了？不是，注释掉仍然起作用，猜想是进行图片的transform会释放？？

---
```python
self.data = [parse_data(data_dir, path, train=self._train) 
             for path in os.listdir(data_dir)]
```

```python
def parse_data(data_dir, path, train=True, one_hot=False):
    # path 是指对任意一个path的图片
    label = int(path.split('.')[0] == 'dog') if train else -1
    # 把label独热编码
    if one_hot:
        label = np.array(map(one_hot_encoding, label))
        
    # 再返回PILimage 典型路径为 storage/Data/cat.6089.jpg
    image = Image.open(os.path.join(data_dir, path))
    return image, label 
```   

则 `self.data` 将会返回[元组列表]， 每个元组是图像和它对应的label  
+ 和官方文档中的 `def find_classes(dir):`+ `def make_dataset(dir, class_to_idx, extensions): ` 实现相同的功能

这样我们的**数据集就创建好了**，具体的使用方法：(在小数据集valid中使用)

In [4]:
import torch 
import torchvision.transforms as transforms

transform_valid = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

main_dir = r'C:\Users\123\Desktop\数据集仓库\dogs-vs-cats'

valid_dataset = MyDogCatData(main_dir, data_type='valid', transform=transform_valid)

valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=50, shuffle=False)


In [5]:
print(len(valid_loader)) # 1是因为这个集合就放了10个图片，1个batch就读完了

1


+ 扩展

[好文：PyTorch自定义数据集](https://www.cnblogs.com/picassooo/p/12846617.html)

```python
class DogVsCatDataset(Dataset):   # 创建一个叫做DogVsCatDataset的Dataset，继承自父类torch.utils.data.Dataset
    def __init__(self, root_dir, train=True, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.img_path = os.listdir(self.root_dir)
        if train:
            self.img_path = list(filter(lambda x: int(x.split('.')[1]) < 10000, self.img_path))    # 划分训练集和验证集
        else:
            self.img_path = list(filter(lambda x: int(x.split('.')[1]) >= 10000, self.img_path))
        self.transform = transform
 
    def __len__(self):
        return len(self.img_path)
 
    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.root_dir, self.img_path[idx]))
        label = 0 if self.img_path[idx].split('.')[0] == 'cat' else 1        # label, 猫为0，狗为1
        if self.transform:
            image = self.transform(image)
        label = torch.from_numpy(np.array([label]))
        return image, label
```
---
### 一个小问题   
他这里要求是：把猫狗数据集的其中前10000张猫的图片和10000张狗的图片作为训练集，把后面的2500张猫的图片和2500张狗的图片作为验证集。猫的label记为0，狗的label记为1。     

+  `__init__`中的 `root_dir` 与 `transform` 与我们这里定义一样，但是我们还定义了 `data_type='train'` 这个参数，没太看懂有什么用。

+ 这里通过 `train=True` 指向(分割已有数据集)得到训练数据集，反之 `train=False`得到验证数据集；

+ 我们前面熟悉的是先得到`train_dataset`再自己分割，而这里是可以直接通过函数`DogVsCatDataset(..train=False,)` 获得 valid_dataset

+ 这里 `self._train = True` -> `mage, label = parse_data(data_dir, path, train=self._train)` -> 最初的读取图像函数中   



```python
def parse_data(data_dir, path, train=True, one_hot=False):
    label = int(path.split('.')[0] == 'dog') if train else -1
    ...
```

+ 如果 设置`if self.data_type == 'train' or self.data_type == 'valid':` 则if train 就是 True，则可以得到解析的图片路径的图片的标签为0(dog)或1(非dog);
+ 而只有我们在使用 `MyDogCatData(data_type='test)` 生成数据集时所有图片的label都是-1
+ 这么做有什么意义？(因为没有完整完成这个练习尚不能确定)


---



### 6.3.2 迁移学习  

有多种方法可以完成此任务的转移学习，例如
1. 也许基于CIFAR10数据训练的model已经很好，使得我们不需要使用Dog-Cat数据训练了。即直接使用CIFAR10 ResNet模型输出狗和猫的分类概率，比较这个概率，然后将图像分配到概率更大的类别中。我们称这种简单的方法为: **inference**，因为我们在这里不训练任何东西，只需进行推理.

2. 也许我们想使用 trained model 作为相同架构的良好起点(除了输出层，因为现在我们只有两个类，而不是十个)。以及不是对我们的相同的架构(例如ResNet)随机初始化参数, 而是使用通过CIFAR10数据训练模型的得到的参数(称为pretrained model), 再在新Dog-Cat数据集(称为 fine-tuning)**训练**, 然后使用 new model 做了分类。我们称这个方法为**fine-tuning**。   

3. 也许我们ResNet的Residual blocks(包含卷积层)在捕获features方面是如此出色，我们只需要学习一些好的方法来在我们新的Dog-Cat任务中组合这些features. 我们可以通过保持使用CIFAR10训练后的model的层中所有其他参数（也称为冻结层）不变（最后一个完全连接的层负责合并提取的特征），并在新的Dog-Cat数据集上进行训练，来完成此操作。预计它将比 fine-tuning 方法快得多。 我们称这种**freeze方法**。   

现在，我们将实现所有上述方法。对于以上方法，我们都需要知道如何 save and load the checkpoints(state_dict)。加载checkpoints:的方式有两种：使用模型的state_dict（包含模型的可学习参数），或使用具有完整信息的整个模型。
下面是我认为对这两种方式最好的解释:https://www.kdnuggets.com/2019/03/deploy-pytorch-model-production.html

1. 直接inference   
+ 在大数据集train中导入数据集

In [6]:
import torch
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((32,32)),  # made it to 32x32
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
main_dir = r'C:\Users\123\Desktop\数据集仓库\dogs-vs-cats'

train_dataset = MyDogCatData(main_dir,
                        data_type="train",
                        transform=transform)

备注：这里自定义读取数据集的时候，在`__init__()`中写读取图片的函数部分导致了读取速度很慢？别的是在`__getitem()__`中写的，是这个原因吗？

+ 迁移学习之直接使用ResNet，在本文出处CNN-Tutorial中ResNet是自己训练的，因为没有做那一部分，我们instead直接调用torchvision中的与训练网络
+ 此处实现对valid_loader的模型预测精度

In [7]:
from torchvision import models
ResNet = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\123/.cache\torch\hub\checkpoints\resnet50-19c8e357.pth


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#net_infer = ResNet().to(device)
#net_infer.load_state_dict(torch.load('storage/resnet.pth'))
net_infer = ResNet.to(device)

net_infer.eval()


with torch.no_grad():
    correct = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net_infer(images)
        
        # Get the softmax output for dogs and cats
        cat_probs = outputs[:,3]
        dog_probs = outputs[:,5]
        
        # Compare and assign the class whose softmax output is larger
        correct += torch.sum((dog_probs >= cat_probs) == labels.to(dtype=torch.uint8))
        

accuracy = int(correct.cpu().numpy()) * 100 / len(valid_dataset)
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 63.64%


2. 迁移学习之**Fine-tuning Method**      
训练了什么？
> 基于所有参数再使用猫狗数据集训练一遍，不冻结参数

In [9]:
import torch
import torchvision.transforms as transforms

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=50, shuffle=True)

+ 定义网络
> load 前面训练过的 resnet网络

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

num_epochs = 30
learning_rate = 0.0001

torch.manual_seed(1111)
np.random.seed(1111)

net_ft = torch.load('storage/resnet.ckpt')

in_feats = net_ft.fc.in_features
### 二分类
net_ft.fc = nn.Linear(in_feats, 2)
net_ft = net_ft.to(device)

criterion = nn.CrossEntropyLoss()
new_optimizer = torch.optim.Adam(net_ft.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    running_loss = 0.0
    val_running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net_ft(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        new_optimizer.zero_grad()
        loss.backward()
        new_optimizer.step()
        
        running_loss += loss.item()

        print ("Epoch [{}/{}], Training Loss: {:.4f}"
               .format(epoch+1, num_epochs, loss.item()))
            
# Save the model checkpoint
torch.save(net_ft.state_dict(), 'storage/resnet2.pth')
torch.save(net_ft, 'storage/resnet2.ckpt')


3. 第三种是冻结一部分参数 -- 在这里，我们通过将require_grad设置为False来访问和冻结所需的参数/层，并仅更新希望他们学习的参数。
> 全连接层不冻结，会被 新的数据集训练


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

num_epochs = 30
learning_rate = 0.01


torch.manual_seed(1111)

net_fz = torch.load('storage/resnet.ckpt')

# Freeze all the parameters,
# excepts the last fc layer 
# (which will be added)
for param in net_fz.parameters():
    param.requires_grad = False

in_feats = net_fz.fc.in_features
net_fz.fc = nn.Linear(in_feats, 2)

net_fz = net_fz.to(device)

new_optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
new_optimizer = torch.optim.Adam(net_fz.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    running_loss = 0.0
    val_running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net_fz(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        new_optimizer.zero_grad()
        loss.backward()
        new_optimizer.step()
        
        running_loss += loss.item()

        print ("Epoch [{}/{}], Training Loss: {:.4f}"
               .format(epoch+1, num_epochs, loss.item()))

# Save the model checkpoint
torch.save(net_fz.state_dict(), 'storage/resnet3.pth')
torch.save(net_fz, 'storage/resnet3.ckpt')



在这种情况下，冻结方法的性能不如微调，但其训练要快得多。在其他情况下将以更少的训练参数实现类似的效果

**本节参考的是00中CNN-Tutorial中的后半部分，但没有学习Resnet部分，有时间可以进一步学习**

+ 将数据集移到数据仓库，防止占用同步内存。